# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [ ]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
  # Load the raw CIFAR-10 data
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # Subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]

  return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for the bonus section.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [ ]:
def identity(im):
    return im.ravel()

In [ ]:
from cs231n.features import *

num_color_bins = 64 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

In [ ]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [1e5, 1e6, 1e7]

results = {}
best_val = -1
best_svm = None

iters = 1500

for rate in learning_rates:
    for strength in regularization_strengths:
        print("learning rate",rate)
        print("regularization strength", strength)
        svm = LinearSVM()
        
        #train it
        loss_hist = svm.train(X_train_feats, y_train, learning_rate=rate, reg=strength, num_iters=iters, verbose=True, batch_size=200)
        
        y_train_pred = svm.predict(X_train_feats)
        print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
        y_val_pred = svm.predict(X_val_feats)
        print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))
        
        accuracy_train = np.mean(y_train == y_train_pred)
        accuracy_valid = np.mean(y_val == y_val_pred)
        results[(rate, strength)] = (accuracy_train, accuracy_valid)
        
        if best_val < accuracy_valid:
            best_val = accuracy_valid
            best_svm = svm
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Evaluate your trained SVM on the test set
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
from cs231n.features import *

num_color_bins = 32 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins), identity]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [ ]:
# Use the validation set to tune the learning rate and regularization strength

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [1e5, 1e6, 1e7]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################
iters = 1500

for rate in learning_rates:
    for strength in regularization_strengths:
        print("learning rate",rate)
        print("regularization strength", strength)
        svm = LinearSVM()
        
        #train it
        loss_hist = svm.train(X_train_feats, y_train, learning_rate=rate, reg=strength, num_iters=iters, verbose=True, batch_size=200)
        
        y_train_pred = svm.predict(X_train_feats)
        print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
        y_val_pred = svm.predict(X_val_feats)
        print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))
        
        accuracy_train = np.mean(y_train == y_train_pred)
        accuracy_valid = np.mean(y_val == y_val_pred)
        results[(rate, strength)] = (accuracy_train, accuracy_valid)
        
        if best_val < accuracy_valid:
            best_val = accuracy_valid
            best_svm = svm
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Evaluate your trained SVM on the test set
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=True)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

### Inline question 1:
The mistakes encountered make sense when we consider input required to create the features. The color histogram feature creates a bias towards images with similar backgrounds or foregrounds as the same class. For example, a deer and other animals photographed against a green/brown background are commonly mis-classified. Similar, the HOG features cause us to put images with similar edges into the same category. The hard edges between car, truck, plane, and ship are quite similar, so the errors make sense.

## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [ ]:
print(X_train_feats.shape)

In [ ]:
from cs231n.classifiers.neural_net import TwoLayerNet
from copy import deepcopy

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

hidden_size_set = [16, 32, 64, 128, 256, 512, 1024]
learning_rate   = [ 5e-2, 3e-2, 1e-2, 1e-3]
learning_rate_decay = [0.99, 0.97, 0.95, 0.90 ]
regularization_strength = [ 0.25, 0.5, 1.0, 1.5]

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
net.train(X_train_feats, y_train, X_val_feats, y_val,
          num_iters=600, batch_size = 200, learning_rate=5e-2,
          learning_rate_decay=0.95, reg=0.0, verbose=True)
best_net = deepcopy(net)
best_acc = (net.predict(X_val_feats) == y_val).mean()

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
for hs in hidden_size_set:
    for lr in learning_rate:
        net = TwoLayerNet(input_dim, hs, num_classes)

        # Train the network
        stats = net.train(X_train_feats, y_train, X_val_feats, y_val,
                num_iters=1000, batch_size=400,
                learning_rate=lr, learning_rate_decay=0.95,
                reg=0.05, verbose=True)

        # Predict on the validation set
        val_acc = (net.predict(X_val_feats) == y_val).mean()
        print("hs=%d,lr=%f"%(hs,lr))
        print('Validation accuracy: ', val_acc)
        #print(net.best_val_acc)
        if val_acc > best_acc:
            best_acc = val_acc
            best_net = deepcopy(net)
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

In [ ]:
# Run your neural net classifier on the test set. You should be able to
# get more than 55% accuracy.

test_acc = (best_net.predict(X_test_feats) == y_test).mean()
print(test_acc)

# Bonus: Design your own features!

You have seen that simple image features can improve classification performance. So far we have tried HOG and color histograms, but other types of features may be able to achieve even better classification performance.

For bonus points, design and implement a new type of feature and use it for image classification on CIFAR-10. Explain how your feature works and why you expect it to be useful for image classification. Implement it in this notebook, cross-validate any hyperparameters, and compare its performance to the HOG + Color histogram baseline.

In [ ]:
"""Compute the difference of Gaussian Filter feature for an image
     
        Uses scipy.ndimage.filters.gaussian_filter;

      Parameters:
        im : an input grayscale or rgb image
        i  : size of the standard deviation of the first gaussian_filter
        j  : size of the standard deviation of the second gaussian_filter

      Returns:
        feat: DoG 
  """
def dog_ij_feature(im,i=1,j=3,order=0):
    if i == j:
        raise ValueError("i=%d must not be equal to j=%d" % (i,j))

    from scipy.ndimage.filters import gaussian_filter
    if im.ndim == 3:
        image = rgb2gray(im)
    else:
        image = np.at_least_2d(im)
    sx, sy = image.shape # image size
    image_gaussian_i = gaussian_filter(image, sigma=i, order=order, mode='reflect', truncate=3.0)
    image_gaussian_j = gaussian_filter(image, sigma=j, order=order, mode='reflect', truncate=3.0)
    
    return (np.abs(image_gaussian_i - image_gaussian_j)/np.max(image_gaussian_i - image_gaussian_j)).ravel()

In [ ]:
def sobel_feature(im, axis=-1):
    if im.ndim == 3:
        image = rgb2gray(im)
    else:
        image = np.at_least_2d(im)
    from scipy.ndimage.filters import sobel
    return sobel(image, axis=axis).ravel()

In [ ]:
plt.figure(figsize=(1.5,1.5,))
plt.axis('off')
plt.imshow(dog_ij_feature(X_train[20],0.01,5,0).reshape(32,32))
test_image = dog_ij_feature(X_train[20],0.01,5,0)
#test_image[test_image < 0.75] = 0
plt.imshow(test_image.reshape(32,32))

In [ ]:
import seaborn as sns

def dog_ij_feature_threshold(im, i=1, j=3, order=0, threshold=0.5):
    if i == j:
        raise ValueError("i=%d must not be equal to j=%d" % (i,j))

    from scipy.ndimage.filters import gaussian_filter
    if im.ndim == 3:
        image = rgb2gray(im)
    else:
        image = np.at_least_2d(im)
    sx, sy = image.shape # image size
    image_gaussian_i = gaussian_filter(image, sigma=i, order=order, mode='reflect', truncate=3.0)
    image_gaussian_j = gaussian_filter(image, sigma=j, order=order, mode='reflect', truncate=3.0)
    
    dog = (np.abs(image_gaussian_i - image_gaussian_j)/np.max(image_gaussian_i - image_gaussian_j)).ravel()
    mean     = np.mean(dog)
    std_dev  = np.std(dog)
    dog[dog < mean - std_dev * threshold] = 0
    return dog


def dog_ij_top_n_pixels(im, i=1, j=3, order=0, n_pixels = 20):
    idx = im.argsort()[:n_pixels]
    return idx


n_pixels = dog_ij_top_n_pixels(dog_ij_feature(X_train[20],0.01,5,0),n_pixels = 80)
print(np.mean(np.unravel_index(list(n_pixels),(32,32)),axis=1))
print(np.std(np.unravel_index(list(n_pixels),(32,32)),axis=1))

print(np.unravel_index(list(n_pixels),(32,32)))
sns.distplot(np.unravel_index(list(n_pixels),(32,32))[0], hist=False, rug=True, kde_kws = {"bw" :2.0})
sns.distplot(np.unravel_index(list(n_pixels),(32,32))[1], hist=False, rug=True, kde_kws = {"bw" :2.0})

plt.figure()
sns.kdeplot(np.unravel_index(list(n_pixels),(32,32))[0], np.unravel_index(list(n_pixels),(32,32))[1], shade=True)

In [66]:
plt.figure(figsize=(1.5,1.5,))
plt.axis('off')
plt.imshow(dog_ij_feature(X_train[20],1,5,0).reshape(32,32))
test_image = dog_ij_feature_threshold(X_train[20],0.01,5,0,0.5)
#test_image[test_image < 0.75] = 0
plt.imshow(dog_ij_feature(X_train[20],0.1,5,0).reshape(32,32))

SyntaxError: invalid syntax (<ipython-input-66-6f719179e65e>, line 1)

In [ ]:
plt.figure(figsize=(1.5,1.5,))
plt.axis('off')
plt.imshow(dog_ij_feature(X_train[20],1,3,0).reshape(32,32))

In [ ]:
plt.figure(figsize=(1.5,1.5,))
plt.axis('off')
plt.imshow(sobel_feature(X_train[20]).reshape(32,32))

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(dog_ij_feature(X_train[idx].astype('uint8'),i=1,j=3,order=0).reshape(32,32))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        transformed_image = dog_ij_feature(X_train[idx].astype('uint8'),i=0.25,j=5,order=0)
        plt.imshow(transformed_image.reshape(32,32))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        transformed_image = dog_ij_feature(X_train[idx].astype('uint8'),i=0.25,j=5,order=0)
        transformed_image[transformed_image < np.mean(transformed_image) - np.std(transformed_image)] = 0
        plt.imshow(transformed_image.reshape(32,32))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(dog_ij_feature(X_train[idx].astype('uint8'),i=1,j=5,order=0).reshape(32,32))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(sobel_feature(X_train[idx].astype('uint8')).reshape(32,32))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
def generate_idx(image, count = 64):
    size = len(image.ravel())
    mask = np.random.random_integers(0, size, count)
    return mask

def sample_image(image, idx):
    return image[idx]

def single_color_image(im, color=0):
    if im.ndim != 3:
        raise ValueError("Invalid input image. Expected a numpy array with dim 3 (bgr).")
    if color < 0 or color > 3:
        raise ValueError("Color should be 0,1,2 (red, green, blue)")
    return im.ravel()[color::3]

def single_channel_image(im, color=0):
    if im.ndim != 3:
        raise ValueError("Invalid input image. Expected a numpy array with dim 3 (bgr).")
    if color < 0 or color > 3:
        raise ValueError("Color should be 0,1,2 (red, green, blue)")
    channel_reduced = np.zeros(im.shape)
    channel_reduced[:,:,color] = single_color_image(im, color).reshape((im.shape[0],im.shape[1]))
    #return the image back in the same 3-dim format
    return channel_reduced

def patch_ravel(img, size=3):
    from sklearn.feature_extraction import image
    patches = image.extract_patches_2d(img, (size,size))
    list_of_patches = []
    for r in patches:
        list_of_patches.append(r.reshape(-1))
    return np.array(list_of_patches).ravel()

for x in list(range(2,30)):
    p = patch_ravel(X_train[1], x)
    print(p.shape)
    del p
    
#shape is (number of windows) x (3 * size * size)
#dimensionality of patch_ravel is too big for this laptop

In [ ]:
feature_fns = [hog_feature,
               lambda img: color_histogram_hsv(img, nbin=num_color_bins),
               lambda image:  dog_ij_feature(image)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats_patch -= mean_feat
X_val_feats_patch -= mean_feat
X_test_feats_patch -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats_patch /= std_feat
X_val_feats_patch /= std_feat
X_test_feats_patch /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])


In [ ]:
from cs231n.classifiers.neural_net import TwoLayerNet
from copy import deepcopy

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

hidden_size_set = [16, 32, 64, 128, 256, 512, 1024]
learning_rate   = [ 5e-2, 3e-2, 1e-2, 1e-3]
learning_rate_decay = [0.99, 0.97, 0.95, 0.90 ]
regularization_strength = [ 0.25, 0.5, 1.0, 1.5]

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
net.train(X_train_feats_patch, y_train, X_val_feats_patch, y_val,
          num_iters=600, batch_size = 200, learning_rate=1e-2,
          learning_rate_decay=0.95, reg=0.0, verbose=True)
best_net = deepcopy(net)
best_acc = (net.predict(X_val_feats) == y_val).mean()


In [ ]:
single_color_image(X_train[8],0)
test = np.zeros((32,32,3))
test[:,:,2] = single_color_image(X_train[5],2).reshape(32,32)
#print(test)
#plt.imshow(test.astype('uint8'))
print(X_train[8].shape)
#plt.imshow(single_channel_image(X_train[5],0))
plt.imshow(X_train[5].astype('uint8'))
print(single_channel_image(X_train[3],1).ndim)
# plt.figure()
# plt.imshow(dog_ij_feature(single_channel_image(X_train[5],0)).reshape(32,32))
# plt.figure()
# plt.imshow(dog_ij_feature(single_channel_image(X_train[5],1)).reshape(32,32))
# plt.figure()
# plt.imshow(dog_ij_feature(single_channel_image(X_train[5],2)).reshape(32,32))

plt.figure()
channel_isolated_DOG = np.zeros(X_train[5].shape)
channel_isolated_DOG[:,:,0] = dog_ij_feature(single_channel_image(X_train[5],0),1,3).reshape(32,32)
channel_isolated_DOG[:,:,1] = dog_ij_feature(single_channel_image(X_train[5],1),1,3).reshape(32,32)
channel_isolated_DOG[:,:,2] = dog_ij_feature(single_channel_image(X_train[5],2),1,3).reshape(32,32)
plt.imshow(channel_isolated_DOG)

plt.figure()
gray_isolated_DOG = rgb2gray(channel_isolated_DOG)
plt.imshow(np.abs(gray_isolated_DOG - dog_ij_feature(X_train[5]).reshape(32,32)))


single_channel_dog = dict()
for k in range(3):
    single_channel_dog[k] = dog_ij_feature(single_channel_image(X_train[5],k),i=1,j=3,order=1)

for k in range(3):
    print(np.max(single_channel_dog[k]),np.mean(single_channel_dog[k]))


In [ ]:
num_color_bins = 16 # Number of bins in the color histogram

feature_fns = [hog_feature,
               lambda img: color_histogram_hsv(img, nbin=num_color_bins),
               lambda image:  dog_ij_feature(image),
               lambda image0: dog_ij_feature(single_channel_image(image0,0)), 
               lambda image1: dog_ij_feature(single_channel_image(image1,1)),
               lambda image2: dog_ij_feature(single_channel_image(image2,2)), 
               identity]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: Add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

In [ ]:
from cs231n.classifiers import LinearSVM
import time
svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train_feats, y_train, learning_rate=1e-7, reg=5e4, num_iters=5000, verbose=True)
toc = time.time()
print('That took %fs' % (toc - tic))
# A useful debugging strategy is to plot the loss as a function of
# iteration number:
plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')

In [ ]:
y_test_pred = svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

# Bonus: Do something extra!
Use the material and code we have presented in this assignment to do something interesting. Was there another question we should have asked? Did any cool ideas pop into your head as you were working on the assignment? This is your chance to show off!